In [0]:
# import tensorflow as tf

In [0]:
# tf.gfile.MkDir('data')
!mkdir data
!mkdir results

In [0]:
# Deep Convolutional GANs

# Importing the libraries
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable

In [14]:
use_cuda = True # True if you want to use CUDA otherwise False

cuda_available = False
device = torch.cuda.get_device_name(0)

if use_cuda and torch.cuda.is_available():
  cuda_available = True
  print('CUDA is available')
else:
  print('CUDA not available')

CUDA is available


In [15]:

# Setting some hyperparameters
batchSize = 64 # We set the size of the batch.
imageSize = 64 # We set the size of the generated images (64x64).

# Creating the transformations
transform = transforms.Compose([transforms.Scale(imageSize), transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),]) # We create a list of transformations (scaling, tensor conversion, normalization) to apply to the input images.

# Loading the dataset
dataset = dset.CIFAR10(root = './data', download = True, transform = transform) # We download the training set in the ./data folder and we apply the previous transformations on each image.
dataloader = torch.utils.data.DataLoader(dataset, batch_size = batchSize, shuffle = True, num_workers = 2) # We use dataLoader to get the images of the training set batch by batch.

# Defining the weights_init function that takes as input a neural network m and that will initialize all its weights.
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

# Defining the generator

class G(nn.Module):

    def __init__(self):
        super(G, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(100, 512, 4, 1, 0, bias = False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias = False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias = False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias = False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 3, 4, 2, 1, bias = False),
            nn.Tanh()
        )

    def forward(self, input):
        output = self.main(input)
        return output

# Creating the generator
netG = G()
netG.apply(weights_init)

if cuda_available:
  netG = netG.cuda()
  print('netG is cuda')

# Defining the discriminator

class D(nn.Module):

    def __init__(self):
        super(D, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1, bias = False),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(64, 128, 4, 2, 1, bias = False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(128, 256, 4, 2, 1, bias = False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(256, 512, 4, 2, 1, bias = False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(512, 1, 4, 1, 0, bias = False),
            nn.Sigmoid()
        )

    def forward(self, input):
        output = self.main(input)
        return output.view(-1)

# Creating the discriminator
netD = D()
netD.apply(weights_init)

if cuda_available:
  netD.cuda()
  print('netD is cuda')


/usr/local/lib/python3.6/dist-packages/torchvision/transforms/transforms.py:209: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


Files already downloaded and verified
netG is cuda
netD is cuda


In [0]:
import time

# Training the DCGANs

criterion = nn.BCELoss()
optimizerD = optim.Adam(netD.parameters(), lr = 0.0002, betas = (0.5, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr = 0.0002, betas = (0.5, 0.999))
epochs = 10

for epoch in range(epochs):
    start_time = time.time()
    
    istart_time = time.time()
    for i, data in enumerate(dataloader, 0):
        
        # 1st Step: Updating the weights of the neural network of the discriminator

        netD.zero_grad()
        
        # 1.1 Training the discriminator with a real image of the dataset
        real, _ = data
        input = Variable(real)
        target = Variable(torch.ones(input.size()[0]))
        
        if cuda_available:
          input = input.cuda()
          target = target.cuda()
        
        output = netD(input)
        errD_real = criterion(output, target)
        
        # 1.2 Training the discriminator with a fake image generated by the generator
        noise = Variable(torch.randn(input.size()[0], 100, 1, 1))
        target = Variable(torch.zeros(input.size()[0]))
        
        
        if cuda_available:
          noise = noise.cuda()
          fake = netG(noise)
          fake = fake.cuda()
          target = target.cuda()
        else:
          fake = netG(noise)
        
        
        output = netD(fake.detach())
        errD_fake = criterion(output, target)
        
        # 1.3 Backpropagating the total error
        errD = errD_real + errD_fake
        errD.backward()
        optimizerD.step()

        # 2nd Step: Updating the weights of the neural network of the generator

        netG.zero_grad()
        target = Variable(torch.ones(input.size()[0]))
        
        if cuda_available:
          target = target.cuda()
          
        output = netD(fake)
        errG = criterion(output, target)
        errG.backward()
        optimizerG.step()
        
        # 3rd Step: Printing the losses and saving the real images and the generated images of the minibatch every 100 steps

        
        if i % 10 == 0:
            iend_time = time.time()
            print('10 processing took: '+str(iend_time - istart_time))  
            istart_time = time.time()
            
            print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f' % (epoch, epochs, i, len(dataloader), errD.data, errG.data))
            print('==============')
            vutils.save_image(real, '%s/real_samples.png' % "./results", normalize = True)
            fake = netG(noise)
            vutils.save_image(fake.data, '%s/fake_samples_i_%03d.png' % ("./results", epoch), normalize = True)
    
    end_time = time.time()
    print('Epoch: '+epoch+' took: '+str(end_time - start_time))  

10 processing took: 0.33267784118652344
[0/10][0/782] Loss_D: 1.6861 Loss_G: 7.0251
10 processing took: 2.2218921184539795
[0/10][10/782] Loss_D: 1.2998 Loss_G: 11.0887
10 processing took: 2.162893295288086
[0/10][20/782] Loss_D: 0.2288 Loss_G: 9.2538
10 processing took: 2.15584135055542
[0/10][30/782] Loss_D: 0.5362 Loss_G: 18.0878
10 processing took: 2.1461341381073
[0/10][40/782] Loss_D: 0.3139 Loss_G: 24.4209
10 processing took: 2.1541285514831543
[0/10][50/782] Loss_D: 0.6218 Loss_G: 27.4326
10 processing took: 2.15041184425354
[0/10][60/782] Loss_D: 0.1236 Loss_G: 7.5171
10 processing took: 2.1684958934783936
[0/10][70/782] Loss_D: 0.1642 Loss_G: 9.8678
10 processing took: 2.1568715572357178
[0/10][80/782] Loss_D: 0.3764 Loss_G: 6.2947
10 processing took: 2.154702663421631
[0/10][90/782] Loss_D: 0.2038 Loss_G: 6.1569
10 processing took: 2.1623799800872803
[0/10][100/782] Loss_D: 0.2165 Loss_G: 4.6030
10 processing took: 2.1605687141418457
[0/10][110/782] Loss_D: 0.2091 Loss_G: 4.